In [1]:
library(Seurat)
library(future)
library(future.apply)
library(dplyr)
library(ggplot2)
library(Matrix)
library(htmlwidgets)
library(plotly)
library(monocle)
library(FNN)
library(reshape2)
library(scales)
library(dplyr)
library(gplots)
library(viridis)
library(pheatmap)

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘htmlwidgets’


The following object is masked from ‘package:SeuratObject’:

    JS


The following object is masked from ‘package:Seurat’:

    JS



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplica

In [2]:
#all data
#####################################################
### Function: finding ancestor node for each node ###
#####################################################
rm(list=ls())
createLineage_Knn <- function(emb, 
                              pd, 
                              reduction="umap", 
                              replication_times=500, 
                              removing_cells_ratio=0.2, 
                              min_cell = 3,#正常3个
                              k_neigh = 5
                              ){
  print(dim(emb))
  if(!"Anno" %in% names(pd) | !"day" %in% names(pd)) {print("Error: no Anno or day in pd")}
  if(sum(rownames(pd)!=rownames(emb))!=0) {print("Error: rownames are not matched")}
  pd$state = pd$Anno
  res = list()
  rep_i = 1
  while(rep_i < (replication_times+1)){
    sampling_index = sample(1:nrow(pd),round(nrow(pd)*(1-removing_cells_ratio)))
    emb_sub = emb[sampling_index,]
    pd_sub = pd[sampling_index,]
    irlba_pca_res_1 <- emb_sub[as.vector(pd_sub$day)=="pre",]
    irlba_pca_res_2 <- emb_sub[as.vector(pd_sub$day)=="nex",]
    pd_sub1 <- pd_sub[pd_sub$day == "pre",]
    pd_sub2 <- pd_sub[pd_sub$day == "nex",]
    pre_state_min = min(table(as.vector(pd_sub1$state)))
    if (pre_state_min < k_neigh & pre_state_min >= min_cell){
      k_neigh = pre_state_min
      print(k_neigh)
    }
    if (pre_state_min < min_cell){
      next
    }
    neighbors <- get.knnx(irlba_pca_res_1, irlba_pca_res_2, k = k_neigh)$nn.index
    tmp1 <- matrix(NA,nrow(neighbors),ncol(neighbors))
    for(i in 1:k_neigh){
      tmp1[,i] <- as.vector(pd_sub1$state)[neighbors[,i]]
    }
    state1 <- names(table(as.vector(pd_sub1$state)))
    state2 <- names(table(as.vector(pd_sub2$state)))
    tmp2 <- matrix(NA,length(state2),length(state1))
    for(i in 1:length(state2)){
      x <- c(tmp1[as.vector(pd_sub2$state)==state2[i],])
      for(j in 1:length(state1)){
        tmp2[i,j] <- sum(x==state1[j])
      }
    }
    tmp2 <- tmp2/apply(tmp2,1,sum)
    tmp2 <- data.frame(tmp2)
    row.names(tmp2) = state2
    names(tmp2) = state1
    res[[rep_i]] = tmp2
    rep_i = rep_i + 1
  }
  return(res)
}

In [3]:
# 设置工作路径和保存路径
work_path = '/data/work/RNA/RNA_ec/'
save_path = '/data/work/RNA/RNA_ec/'

# 读取汇总的rds文件（包括所有时间点数据）
dat <- readRDS("/data/work/RNA/coembed_germ_layer_ectoderm.Rds")

setwd(work_path)

In [4]:
rna_cells <- rownames(dat@meta.data[dat@meta.data$assay == "RNA", ])
# 2. 创建一个新的 Seurat 对象，只包含这些细胞
ect_rna <- subset(dat, cells = rna_cells)
ect_rna
dat<-ect_rna

An object of class Seurat 
16725 features across 13774 samples within 1 assay 
Active assay: RNA (16725 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap

In [8]:
emb <- data.frame(Embeddings(object = dat, reduction = "umap"))
anno <- dat@meta.data
anno$Anno <- as.vector(anno$celltype_in_coembed)  # 假设Annotation_1_cell_type存储细胞类型
anno$stage <- anno$infer_age_int  # 假设ifer_age存储时间点信息
anno$stage

[1] 14 13 13 13 13 13 16 13 13 13 13 13 16 13 12 12 19 13 14 14 12 13 13 12
   [25] 15 13 14 14 13 13 12 14 13 13 13 13 13 14 14  8 13 13 13 14 15 15 12 15
   [49] 13 13 14 14 14 13 19 13 13 13 13 12 13 16 13 16 13 15 14 14 15 13 13 14
   [73] 13 13 13 13 16 13 12 13 13 13 13 13 14 13 13 18 14 15 13 13 13 13 14 14
   [97] 17 13 14 13 13 13 13 13 14 12 13 13 13 13 13 13 13 15 13 14 13 13 13 13
  [121] 13 13 15 16 15 13 12 14 19 13 13 13 13 13 13 13 13 13 13 18 14 17 12 18
  [145] 13 13 13 13 13 13 13 19 13 13 15 13 15 14 14 16 15 12 13 14 12 13 13 15
  [169] 12 13 14 16 13 13 13 15 15 13 12 13 13 15 14 15 15 13 12 15 14 14 14 16
  [193] 13 12 13 18 11 13 13 13 14 19 15 14 13 15 17 16 15 14 14 14 16 19 13 15
  [217] 15 14 16  9 13 15 13 14 13 13 18 13 13 13 14 14 12 16 18 13 16 12 13 13
  [241] 13 12 13 15 13 13 14 13 13 16 16 16 13 13 13 18 12 14 15 12 17 13 13 12
  [265] 14 15 14 13 13 15 14 13 13 15 14 15 13 15 15 13 15 18 13 14 15 13 15 15
  [289] 14 16 13 13 15 15 13 13 10 15 13 18 13 13 13 13 15 13 13 13 13 17 13 19
  [313] 13 13 13 13 12 15 15 15 13 14 12 13 13 13 16 13 15 14 18 14 13 12 19 15
  [337] 14 13 15 13 12 14 13 19 13 13 13 12 14 13 14 16 14 14 17 12 12 15 14 15
  [361] 14 13 13 13 16 13 16 13 13 13 13 13 13 13 13 14 13 13 15 14 16 13 13 19
  [385] 16 13 13 16 15 14 13 16 13 13 13 14 18 17 14 16 17 15 13 12 13 13 18 15
  [409] 13 13 13 16 15 13 15 13 14 12 13 13 13 14 19 13 16 13 13 15 13 17 13 15
  [433] 13 15 13 15 14 14 16 15 18 17 16 16 16 13 16 13 13 13 13 16 16 13 13 13
  [457] 13 13 15 15 13 13 14 14 16 14 14 13 14 13 14 13 15 14 13 15 12 13 13 12
  [481] 14 13 18 13 13 14 13 15 13 15 13 15 13 15 13 11 13 15 14 14 13 15 13 13
  [505] 15 14 15 16 13 13 16 13 13 15 13 15 15 13 14 16 13 14 13 13 16 12 13 13
  [529] 14 16 13 13 14 13 13 13 13 13 15 15 15 13 13 13 13 13 15 13 13 15 14 13
  [553] 15 16 17 13 12 13 15 19 16 13 13 17 13 13 15 13 12 13 13 14 18 14 16 12
  [577] 14 16 13 14 17 15 13 13 17 13 14 19 15 14 16 16 14 14 16 15 13 19 16 12
  [601] 16 15 15 16 14 13 14 13 15 13 13 13 14 13 15 13 14 15 16 13 14 13 13 13
  [625] 17 13 13 16 16 13 16 20 15 15 14 13 13 14 15 13 18 16 15 13 13 13 17 13
  [649] 13 16 12 14 14 15 17 13 13 18 15 13 13 16 13 15 18 16 12 13 13 13 12 15
  [673] 14 15 14 14 15 13 16 13 16 13 13 19 13 13 12  9 14 13 15 16 13 15 13 19
  [697] 14 13 14 12 15 15 15 13 13 12 16 13 13 13 14 13 12 10 13 13 13 16 13 13
  [721] 13 13 14 17 15 13 14 15 14 15 13 13 14 15 15 13 16 13 18 16 13 16 16 15
  [745] 14 13 13 11 13 13 13 13 14 13 13 14 13 13 15 14 13 15 13 14 19 13 13 18
  [769] 12 13 18 13 13 13 13 16 18 15 14 16 13 13 16 16 19 15 13 16 15 16 13 18
  [793] 14 15 17 15 14 14 13 14 18 17 16 14 13 15 15 14 18 13 15 14 13 14 14 13
  [817] 15 15 13 14 19 18 13 15 18 13 16 13 15 15 15 15 13 15 13 16 13 13 16 18
  [841] 12 16 14 13 12 13 14 18 15 15 15 13 13 13 13 15 15 13 13 16 13 13 15 15
  [865] 15 15 13 13 13 13 13 16 18 15 13 13 13 18 15 16 15 13 13 12 13 14 13 19
  [889] 14 16 16 15 16 13 14 16 19 13 13 15 16 12 15 13 16 13 12 15 13 15 15 16
  [913] 15 14 17 13 17 19 16 13 13 14 14 13 16 12 13 14 13 14 13 13 16 17 14 16
  [937] 15 16 16 16 14 13 19 13 13 13 13 12 13 14 13 13 13 13 13 15 12 13 16 13
  [961] 13 13 13 12 15 13 14 14 13 13 13 14 13 15 13 12 14 14 13 13 13 15 13 13
  [985] 16 13 13 13 14 14 18 13 13 13 14 16 15 13 14 16 13 12 13 16 13 14 14 13
 [1009] 13 13 13 15 13 13 15 12 13 15 13 13 13 13 13 14 13 13 15 13 14 13 15 14
 [1033] 13 15 13 13 18 13 14 13 13 15 13 13 14 12 13 13 17 13 12 13 13 14 13 15
 [1057] 15 13 13 13 15 19 13 14 13 13 15 16 12 14 15 14 14 16 14 17 13 15 13 14
 [1081] 15 14 14 13 14 13 18 16 16 18 17 16 14 16 13 14 13 15 14 13 13 13 13 14
 [1105] 13 13 18 19 13 13  9 13 13 13 13 14 13 14 13 13 18 14 14 12 14 13 14 13
 [1129] 18 13 13 13 14 19 13 13 13 16 13 13 15 16 13 14 15 14 14 17 14 14 17 19
 [1153] 13 14 13 13 16 19 16 16 13 14 13 13 16 13 14 14 13 14 15 13 13 13 17 14
 [1177] 13 15 14 13 15 14 15 16 14 13 15 13 15 14 14 14 13 13 13 16 13 19 13 14
 [12

In [10]:
time_points <- unique(anno$infer_age_int) 
time_points <- sort(time_points)  # 排序时间点，确保按顺序进行两两计算

In [ ]:
# 循环两两计算KNN谱系
for (i in 1:(length(time_points) - 1)) {
  for (j in (i + 1):length(time_points)) {
    time_1 <- time_points[i]
    time_2 <- time_points[j]
    
    # 筛选对应时间点的数据
    anno1 <- anno[anno$infer_age_int == time_1, ]
    anno2 <- anno[anno$infer_age_int == time_2, ]
    
    # 按照细胞类型进行采样，每个细胞类型取20%
    anno1_sampled <- do.call(rbind, lapply(unique(anno1$Anno), function(cell_type) {
      cell_type_data <- anno1[anno1$Anno == cell_type, ]
      sample_size <- ceiling(0.2 * nrow(cell_type_data))  # 每个细胞类型取20%
      sampled_data <- cell_type_data[sample(1:nrow(cell_type_data), sample_size), ]
      return(sampled_data)
    }))
    
    anno2_sampled <- do.call(rbind, lapply(unique(anno2$Anno), function(cell_type) {
      cell_type_data <- anno2[anno2$Anno == cell_type, ]
      sample_size <- ceiling(0.2 * nrow(cell_type_data))  # 每个细胞类型取20%
      sampled_data <- cell_type_data[sample(1:nrow(cell_type_data), sample_size), ]
      return(sampled_data)
    }))
    
    # 添加处理和阶段信息
    anno1_sampled$day <- 'pre'
    anno2_sampled$day <- 'nex'
    anno1_sampled$stage <- time_1
    anno2_sampled$stage <- time_2
    
    # 获取样本对应的embeddings
    emb1 <- emb[rownames(anno1_sampled), ]
    emb2 <- emb[rownames(anno2_sampled), ]
    
    # 合并数据
    combined_emb <- rbind(emb1, emb2)
    combined_anno <- rbind(anno1_sampled, anno2_sampled)
    
    # 计算KNN谱系
    res <- createLineage_Knn(combined_emb, combined_anno, k_neigh = 5)
    
    # 保存KNN谱系结果
    saveRDS(res, paste0("knn_lineage_", time_1, "_", time_2, ".rds"))
    
    # 测试不同k值的稳定性
    replication_times <- 500
    dat <- res
    state_1 <- row.names(dat[[1]])
    state_2 <- names(dat[[1]])

    # 创建一个空矩阵存储中位数
    tmp_1 <- matrix(NA, nrow(dat[[1]]), ncol(dat[[1]]))

    # 计算每个位置的中位数
    for (i in 1:nrow(dat[[1]])) {
      for (j in 1:ncol(dat[[1]])) {
        xx <- NULL
        for (k in 1:replication_times) {
          xx <- c(xx, dat[[k]][i,j])
        }
        tmp_1[i, j] <- median(xx[!is.na(xx)])
      }
    }

    tmp_1 <- data.frame(tmp_1)
    row.names(tmp_1) <- paste0(state_1)
    names(tmp_1) <- paste0(state_2)

    # 保存中位数数据
    write.csv(tmp_1, paste0("knn_umap_median_", time_1, "_", time_2, ".csv"))
  }
}


In [ ]:
# 设置工作路径和保存路径
work_path = '/data/work/RNA/ATAC_ec/'
save_path = '/data/work/RNA/ATAC_ec/'

# 读取汇总的rds文件（包括所有时间点数据）
dat <- readRDS("/data/work/RNA/coembed_germ_layer_ectoderm.Rds")

setwd(work_path)

In [ ]:
rna_cells <- rownames(dat@meta.data[dat@meta.data$assay == "ATAC", ])
# 2. 创建一个新的 Seurat 对象，只包含这些细胞
ect_rna <- subset(dat, cells = rna_cells)
dat<-ect_rna

In [ ]:
emb <- data.frame(Embeddings(object = dat, reduction = "umap"))
anno_1 <- rna_data@meta.data
anno$Anno <- as.vector(anno_1$celltype_in_coembed)  # 假设Annotation_1_cell_type存储细胞类型
anno$stage <- anno$infer_age_int  # 假设ifer_age存储时间点信息
anno$stage


In [ ]:
# 循环两两计算KNN谱系
for (i in 1:(length(time_points) - 1)) {
  for (j in (i + 1):length(time_points)) {
    time_1 <- time_points[i]
    time_2 <- time_points[j]
    
    # 根据时间点筛选样本
    anno1 <- anno[anno$infer_age_int == time_1, ]
    anno2 <- anno[anno$infer_age_int == time_2, ]
    
    # 按细胞类型分组并进行采样，每个细胞类型取20%
    anno1_sampled <- do.call(rbind, lapply(unique(anno1$cell_type), function(cell_type) {
      cell_type_data <- anno1[anno1$cell_type == cell_type, ]
      sample_size <- ceiling(0.2 * nrow(cell_type_data))  # 每个细胞类型取20%
      sampled_data <- cell_type_data[sample(1:nrow(cell_type_data), sample_size), ]
      return(sampled_data)
    }))
    
    anno2_sampled <- do.call(rbind, lapply(unique(anno2$cell_type), function(cell_type) {
      cell_type_data <- anno2[anno2$cell_type == cell_type, ]
      sample_size <- ceiling(0.2 * nrow(cell_type_data))  # 每个细胞类型取20%
      sampled_data <- cell_type_data[sample(1:nrow(cell_type_data), sample_size), ]
      return(sampled_data)
    }))
    
    # 添加预处理和后续阶段信息
    anno1_sampled$day <- 'pre'
    anno2_sampled$day <- 'nex'
    anno1_sampled$stage <- time_1
    anno2_sampled$stage <- time_2
    
    # 获取样本对应的embeddings
    emb1 <- emb[rownames(anno1_sampled), ]
    emb2 <- emb[rownames(anno2_sampled), ]
    
    # 合并数据
    combined_emb <- rbind(emb1, emb2)
    combined_anno <- rbind(anno1_sampled, anno2_sampled)
    
    # 计算KNN谱系
    res <- createLineage_Knn(combined_emb, combined_anno, k_neigh = 5)
    
    # 保存KNN谱系结果
    saveRDS(res, paste0("knn_lineage_", time_1, "_", time_2, ".rds"))
    
    # 测试不同k值的稳定性
    replication_times <- 500
    dat <- res
    state_1 <- row.names(dat[[1]])
    state_2 <- names(dat[[1]])

    # 创建一个空矩阵存储中位数
    tmp_1 <- matrix(NA, nrow(dat[[1]]), ncol(dat[[1]]))

    # 计算每个位置的中位数
    for (i in 1:nrow(dat[[1]])) {
      for (j in 1:ncol(dat[[1]])) {
        xx <- NULL
        for (k in 1:replication_times) {
          xx <- c(xx, dat[[k]][i,j])
        }
        tmp_1[i, j] <- median(xx[!is.na(xx)])
      }
    }

    tmp_1 <- data.frame(tmp_1)
    row.names(tmp_1) <- paste0(state_1)
    names(tmp_1) <- paste0(state_2)

    # 保存中位数数据
    write.csv(tmp_1, paste0("knn_umap_median_", time_1, "_", time_2, ".csv"))
  }
}
